# OAuth2 Web Application Flow Example

A short example on how to communicate with a protected resource using the OAuth 2 flow for web applications.

## Chosen Library

[Requests-Oauthlib](https://requests-oauthlib.readthedocs.io/en/latest/) is the chosen library for this example. It makes use of [Requests](https://requests.readthedocs.io/en/master/) and [OAuthLib](https://oauthlib.readthedocs.io/en/latest/). To include it in your project just add the following import:

In [ ]:
from requests_oauthlib import OAuth2Session

## Initial Configuration

In [ ]:
import os
import pprint

# Disable TLS to use a plain HTTP callback
os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

printer = pprint.PrettyPrinter(indent=2)

# The session is used as a simple DB for this example
session = {}

# Credentials obtained from registering a new application
client_id = "<the id obtained from google>.apps.googleusercontent.com"
client_secret = "<the secret obtained from google>"
redirect_uri = "https://your.registered/callback"

# OAuth endpoints obtained from the Google API documentation
authorization_base_url = "https://accounts.google.com/o/oauth2/v2/auth"
token_url = "https://www.googleapis.com/oauth2/v4/token"
refresh_url = token_url # True for Google but not all providers
# Scopes needed to query the chosen protected resource
scope = [
    "openid",
    # User API
    # "https://www.googleapis.com/auth/userinfo.email",
    # "https://www.googleapis.com/auth/userinfo.profile",
    # Directory API
    # "https://www.googleapis.com/auth/admin.directory.group.member.readonly",
    # "https://www.googleapis.com/auth/admin.directory.group",
    # "https://www.googleapis.com/auth/admin.directory.user",
    # Reports API
    "https://www.googleapis.com/auth/admin.reports.audit.readonly",
    "https://www.googleapis.com/auth/admin.reports.usage.readonly",
]

## Authorization

The flow starts by hitting an endpoint defined for this purpose in your back-end service. This example will emulate that by doing all steps manually.

In [ ]:
# Session initialization
provider = OAuth2Session(client_id, scope=scope, redirect_uri=redirect_uri)

# Redirect user to Google for authorization
authorization_url, state = provider.authorization_url(
    authorization_base_url,
    # offline for refresh token
    # force to always make user click authorize
    access_type="offline",
    prompt="consent",
)

# State is used to prevent CSRF, keep this for later
session["oauth_state"] = state

print(f"Please go here and authorize {authorization_url}")

## Callback Simulation

Done when the provider redirects the user the `redirect_url` defined previously. This URL should be configured as an authorized URI in your chosen provider.

### Redirect URL containing the authorization code

In [ ]:
# Get the authorization verifier code from the callback url
authorization_response = input("Paste the full redirect URL here: ")

### Retrieve token with the code from the authorization URL

In [ ]:
provider = OAuth2Session(
    client_id, 
    redirect_uri=redirect_uri,
    state=session["oauth_state"],
)

# Fetch the access token
token = provider.fetch_token(
    token_url, 
    authorization_response=authorization_response,
    # Google specific extra parameter used for client authentication
    client_secret=client_secret,
)

# Store the token for later use
session["oauth_token"] = token

printer.pprint(token)

## Fetching a Protected Resource

Access a protected resource using the OAuth 2 token from the previous step.

In [ ]:
from datetime import datetime, timedelta
from time import time

token = session["oauth_token"]

# Enable this to test the automatic access token refresh feature.
# We force an expiration by setting expired at in the past.
# This will trigger an automatic refresh next time we interact with the provider.
# token["expires_at"] = time() - 86400 # 24 hours

extra = {
    "client_id": client_id,
    "client_secret": client_secret,
}

def token_updater(token):
        session["oauth_token"].update(token)

provider = OAuth2Session(
    client_id,
    token=token,
    auto_refresh_kwargs=extra,
    auto_refresh_url=refresh_url,
    token_updater=token_updater
)

# With a super admin account
# https://developers.google.com/admin-sdk/reports/v1/appendix/usage/customer/accounts
# https://developers.google.com/admin-sdk/reports/v1/guides/manage-usage-customers
parameters = ",".join(
    [
        "accounts:apps_total_licenses",
        "accounts:apps_used_licenses",
        "accounts:gsuite_basic_total_licenses",
        "accounts:gsuite_basic_used_licenses",
        "accounts:gsuite_enterprise_total_licenses",
        "accounts:gsuite_enterprise_used_licenses",
        "accounts:gsuite_unlimited_total_licenses",
        "accounts:gsuite_unlimited_used_licenses",
        "accounts:total_quota_in_mb",
        "accounts:used_quota_in_mb",
        "accounts:team_drive_used_quota_in_mb",
        "accounts:num_users",
        "accounts:num_locked_users",
        "accounts:num_suspended_users",
        "accounts:num_disabled_accounts",
        "accounts:num_users_2sv_enforced",
        "accounts:num_users_2sv_enrolled",
        "accounts:num_users_2sv_enrolled_and_enforced",
        "accounts:num_users_2sv_not_enforced",
        "accounts:num_users_2sv_not_enrolled",
        "accounts:num_users_2sv_not_enrolled_but_enforced",
        "accounts:num_7day_logins",
        "accounts:num_30day_logins",
    ]
)
# Data for Reports API lags behind up to a maximum of three days and a minimum of one 
# but there's no way to know if data is there unless you call the API
n_days_ago = datetime.today() - timedelta(days=3)
n_days_ago_str = n_days_ago.strftime("%Y-%m-%d")
r = provider.get(
    f"https://www.googleapis.com/admin/reports/v1/usage/dates/{n_days_ago_str}?parameters={parameters}",
).json()

printer.pprint(r)
print()
printer.pprint(token)
